In [4]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import pickle

In [5]:
page = requests.get('https://maldita.es/malditobulo/')

In [6]:
#print ("pagina", page)
print ("pagina status code", page.status_code)
#print ("contenido de la pagina", page.content)

pagina status code 200


In [7]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
#print(soup.prettify())

In [8]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces

In [9]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
    #print(title)

    date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="article-body")
    body2 = body.find_all("p", class_="")

    for e in body2:
        content_list.append(e.get_text().replace("\xa0", " "))
    
    content = ''.join(content_list)
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url, "Fuente": "Maldita.es"}
    
    return article

In [ ]:
lista_enlaces = []
suficiente = True
n_pag = 1
n_noticias = 1#Las noticias van en bloques de 24. La carga computacional aumentaria mucho si quisiera que fuera mas preciso
                #Pero para este caso de uso y estudio no es necesario.

#Cargo las URLS.
print('Cargando URLs...')

while (len(lista_enlaces) <= n_noticias-1 and suficiente):
    link =  "https://maldita.es/malditobulo/page/" + str(n_pag) + "/"
    arrayurls= get_urls(link, lista_enlaces)
    n_pag = n_pag + 1

    if (n_pag >=10):
        suficiente=False
        
lista_articulos = []   

for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)
    
print(len(lista_enlaces))
print(lista_enlaces)

Cargando URLs...


In [ ]:
lista_articulos

In [ ]:
from datetime import date


date = date.today().strftime("%d-%m-%Y")


filename = 'Maldita_FN_' + date
print(filename)

outfile = open(filename,'wb')

pickle.dump(lista_articulos, outfile)
outfile.close()

In [ ]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict